Preprocessing From the previous assignment:

In [ ]:
from Data_Preprocessing import prepare_data
import pandas as pd
import numpy as np
import math
from numpy import nan
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
!rm *.csv
from google.colab import files
print("Please load the raw csv data")
uploadedData = files.upload()
filename = list(uploadedData)[0]
dataset = pd.read_csv(filename)
train_data, test_data= train_test_split(dataset, test_size=0.2, train_size=0.8, random_state=156)
dataset, train_data = Data_Preprocessing.prepare_data.prepare_data(dataset, train_data)



In [ ]:
# **Part 1: Basic model selection with k-Nearest Neighbors:**
from visualize_clf import *
# Visualization and basic analysis
# Q1:
temp_df = train_data[['PCR_08', 'PCR_10', 'risk']]
p = sns.jointplot(data = temp_df, x='PCR_08', y='PCR_10', hue = 'risk')
p.fig.suptitle("Marginal and joint distribution of PCR_08 and PCR_10 according to 'risk' label")

# Q2:
from sklearn.neighbors import KNeighborsClassifier
X = temp_df[['PCR_08', 'PCR_10']]
y = temp_df[['risk']]
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)
visualize_clf(knn, X.to_numpy(), y.to_numpy(), 'Decision Reigons - k=1 neighbor', xlabel = 'PCR_08', ylabel = 'PCR_10')

# Q3:
from sklearn.model_selection import cross_validate
k_range = list(range(1, 20, 1)) + list(range(20, 300, 10))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn, X, y, cv=5, return_train_score=True)
    k_scores.append(scores['train_score'].mean())
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()



